# Preprocessing

In [85]:
def preprocessing(file):

    # read txt file
    input = open(file, "r")
    text = input.read()

    # remove numbers and stop words
    no_numbers = ''.join([i for i in text if not i.isdigit()]).lower()

    # remove punctuations and new line char
    import re
    only_text = re.sub(r"[^a-z\s]+", ' ', no_numbers)
    
    # replace multiple consecutive spaces with a single space
    to_tokens = re.sub(r'\s+', ' ', only_text).strip()
    tokens = [s for s in to_tokens.strip().split() if s.strip()]

    return tokens


In [86]:
tokens = preprocessing("misc_files/List-of-Glagolitic-manuscripts-Wikipedia.txt")

# Split

In [87]:
def splitlines(tokens: list, split=50000):
    split1 = tokens[0:split]
    split2 = tokens[split:]

    return split1, split2

# Mapper

In [88]:
def mapper(text,out_queue):
  keyval = []
  for i in text:
    wordssplit = i.split()
    for j in wordssplit:
      keyval.append([j,1])      #Appending each word in the line with 1 and storing it in ["word",1] format in a nested list
  out_queue.put(keyval)

# Sorting

In [89]:
def sortedlists(list1,list2):
  out = list1 + list2             #Appending the two input lists into a single list
  out.sort(key= lambda x :x[0])   #Sorting the lists based on the first element of the list which is the "word"
  return out

# Partition

In [90]:
def partition(sorted_list) :
 sort1out = []
 sort2out = []
 for i in sorted_list:
    if i[0][0] < 'n':             #Partitioning the sorted word list into two separate lists 
      sort1out.append(i)          #with first list containing words starting with a-m and n-z into second
    else : sort2out.append(i)
 return sort1out,sort2out

# Reducer

In [91]:
def reducer(part_out1,out_queue) :
  sum_reduced = []
  count = 1
  for i in range(0,len(part_out1)):
    if i < len(part_out1)-1:
      if part_out1[i] == part_out1[i+1]:
       count = count+1                              #Counting the number of words
      else : 
       sum_reduced.append([part_out1[i][0],count])  #Appending the word along with count to sum_reduced list as ["word",count]
       count = 1 
    else: sum_reduced.append(part_out1[i])          #Appending the last word to the output list    
  out_queue.put(sum_reduced)

In [81]:
import threading
import queue
def multi_thread_function(func,map1_input,map2_input):  #func is the function to be used with two threads taking two inputs map1_input and map2_input
  my_queue1 = queue.Queue()  #Using queue to store the values of mapper output to use them in sort function
  my_queue2 = queue.Queue()
  t1 = threading.Thread(target=func, args=(map1_input,my_queue1)) 
  t2 = threading.Thread(target=func, args=(map2_input,my_queue2))  
  t1.start()                 #Starting the execution of thread1
  t2.start()                 #Starting the execution of thread2 to run simultaneously with thread1
  t1.join()                  #Waiting for the thread1 to be completely executed
  t2.join()                  #Waiting for the thread2 to be completely executed
  list1out = my_queue1.get() #Getting the values from the queue into a variable to return its value
  list2out = my_queue2.get()
  return list1out,list2out

In [95]:
def main_function(text):  
  cleantext = preprocessing(text)
  linessplit = splitlines(cleantext,50000)
  mapperout = multi_thread_function(mapper,linessplit[0],linessplit[1]) 
  sortedwords = sortedlists(mapperout[0],mapperout[1])
  slicedwords = partition(sortedwords)
  reducerout = multi_thread_function(reducer,slicedwords[0],slicedwords[1])
  return reducerout[0]+reducerout[1]

output = main_function("misc_files/List-of-Glagolitic-manuscripts-Wikipedia.txt")